# Один из возможных алгоритмов скрейпинга

* указать в коде адрес интересующего сайта: откуда вы хотите скачать данные?
для этого используется библиотека requests

* сохранить веб-страницу (html-код страницы)

* выбрать данные, которые нужно собрать (используется BeautifulSoup)

* записать данные в csv-файл.

Если нужно соскрейпить несколько страниц - повторяем процесс для каждой из них.

**Наша задача:** выгрузить недавние новости в датафрейм `pandas`, чтобы потом сохранить все в csv-файл.

Сначала сгрузим весь html-код страницы и сохраним его в отдельную переменную. Для этого нам понадобится библиотека `requests`. Документация: https://requests.readthedocs.io/en/latest/

Импортируем её:

In [2]:
!pip install pandas

In [1]:
import requests

Сохраним ссылку на главную страницу сайта в переменную `url` для удобства и выгрузим страницу. (Разумеется, это будет работать при подключении к интернету. Если соединение будет отключено, Python выдаст `NewConnectionError`).

In [3]:
url = 'https://nplus1.ru/' # сохраняем
page = requests.get(url) # загружаем страницу по ссылке

Если мы просто посмотрим на объект, мы ничего особенного не увидим:

In [4]:
page  # response 200 - страница загружена

<Response [200]>

In [5]:
dir(page)

['__attrs__',
 '__bool__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_content',
 '_content_consumed',
 '_next',
 'apparent_encoding',
 'close',
 'connection',
 'content',
 'cookies',
 'elapsed',
 'encoding',
 'headers',
 'history',
 'is_permanent_redirect',
 'is_redirect',
 'iter_content',
 'iter_lines',
 'json',
 'links',
 'next',
 'ok',
 'raise_for_status',
 'raw',
 'reason',
 'request',
 'status_code',
 'text',
 'url']

In [6]:
page.text

'<!DOCTYPE html>\n<html prefix="og: http://ogp.me/ns#" lang="ru">\n<head>\n  <meta charset="utf-8">\n  <meta name="viewport" content="width=device-width, initial-scale=1">\n  <title>N + 1 — главное издание о науке, технике и технологиях</title>\n    <link rel="preload" href="https://staticn1.nplus1.ru/fonts/AeonikPro/AeonikPro-Regular.woff2" as="font" type="font/woff2" crossorigin />\n    <link rel="preload" href="https://staticn1.nplus1.ru/fonts/Spectral/Spectral-Regular.woff" as="font" type="font/woff2" crossorigin />\n  <link href="/front-build/css/main.css?id=9b496fbb252428c03791da99722528ab" rel="stylesheet">\n  <link href="/front-build/css/app.css?id=f9c00b5102d287d01698f43324336ca6" rel="stylesheet">\n  \n\n  <link rel="apple-touch-icon" sizes="180x180" href="/apple-touch-icon.png">\n  <link rel="icon" type="image/png" sizes="32x32" href="/favicon-32x32.png">\n  <link rel="icon" type="image/png" sizes="16x16" href="/favicon-16x16.png">\n  <link rel="manifest" href="/site.webmani

**BeautifulSoup** - python-библиотека для синтаксического разбора файлов HTML/XML. В веб-разработке «суп из тегов» (tag soup) - это слово для синтаксически или структурно некорректного HTML, написанного для веб-страницы.

Документация: https://www.crummy.com/software/BeautifulSoup/bs4/doc/

Импортируем функцию `BeautifulSoup` из библиотеки `bs4` (от *beautifulsoup4*) и заберём со страницы `page` код html в виде текста.

Сохраним в переменную 'soup' весь HTML-код страницы. HTML-код - это "дерево тегов", формирующее контент страницы.

In [7]:
!pip install bs4

  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1256 sha256=ca3f8b1a560a6376e8c2f7659074c86cd715a9f6a02abd6ba7cdea47e2871ffc
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4


In [8]:
from bs4 import BeautifulSoup
# название - отсылка к песне про суп из Алисы в стране чудес https://aliceinwonderland.fandom.com/wiki/Turtle_Soup

In [9]:
soup = BeautifulSoup(page.text, 'html')

Если выведем `soup` на экран, мы увидим то же самое, что в режиме разработчика или в режиме происмотра исходного кода (`view-source` через *Ctrl+U* в Google Chrome).

In [10]:
soup

<!DOCTYPE html>
<html lang="ru" prefix="og: http://ogp.me/ns#">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<title>N + 1 — главное издание о науке, технике и технологиях</title>
<link as="font" crossorigin="" href="https://staticn1.nplus1.ru/fonts/AeonikPro/AeonikPro-Regular.woff2" rel="preload" type="font/woff2"/>
<link as="font" crossorigin="" href="https://staticn1.nplus1.ru/fonts/Spectral/Spectral-Regular.woff" rel="preload" type="font/woff2"/>
<link href="/front-build/css/main.css?id=9b496fbb252428c03791da99722528ab" rel="stylesheet"/>
<link href="/front-build/css/app.css?id=f9c00b5102d287d01698f43324336ca6" rel="stylesheet"/>
<link href="/apple-touch-icon.png" rel="apple-touch-icon" sizes="180x180"/>
<link href="/favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/>
<link href="/favicon-16x16.png" rel="icon" sizes="16x16" type="image/png"/>
<link href="/site.webmanifest" rel="manifest"/>
<link color="#f26e40" href=

Для просмотра выглядит не очень удобно.  «Причешем» наш `soup` – воспользуемся методом `.prettify()` в сочетании с функцией `print()`.

In [11]:
print(soup.prettify())

<!DOCTYPE html>
<html lang="ru" prefix="og: http://ogp.me/ns#">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <title>
   N + 1 — главное издание о науке, технике и технологиях
  </title>
  <link as="font" crossorigin="" href="https://staticn1.nplus1.ru/fonts/AeonikPro/AeonikPro-Regular.woff2" rel="preload" type="font/woff2"/>
  <link as="font" crossorigin="" href="https://staticn1.nplus1.ru/fonts/Spectral/Spectral-Regular.woff" rel="preload" type="font/woff2"/>
  <link href="/front-build/css/main.css?id=9b496fbb252428c03791da99722528ab" rel="stylesheet"/>
  <link href="/front-build/css/app.css?id=f9c00b5102d287d01698f43324336ca6" rel="stylesheet"/>
  <link href="/apple-touch-icon.png" rel="apple-touch-icon" sizes="180x180"/>
  <link href="/favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/>
  <link href="/favicon-16x16.png" rel="icon" sizes="16x16" type="image/png"/>
  <link href="/site.webmanifest" rel="manifest"

В такой выдаче ориентироваться гораздо удобнее (но при желании, то же можно увидеть в браузере, на большом экране).

Чтобы сгрузить все новости с главной страницы сайта, нужно собрать все ссылки на страницы с этими новостями. Ссылки в html-файле всегда заключены в тэг `<a></a>` и имеют атрибут `href`.

**Функция `soup.find('a')`** найдет первый в дереве тег \<a>.
Если нам нужно найти не только первый элемент, а все элементы по определенному признаку, следует использовать функцию **`soup.find_all('a')`**

In [12]:
for link in soup.find_all('a'):
    print(link.get('href'))

/search
https://offline.nplus1.ru/
https://nplus.pro/
https://nplus1.ru/about
https://nplus1.ru/difficult
https://nplus1.ru/adv
https://nplus1.ru/blog/2022/04/01/samotek
https://nplus1.ru/search?tags=946
https://nplus1.ru/search?tags=869
https://nplus1.ru/search?tags=874
https://nplus1.ru/search?tags=880
https://nplus1.ru/search?tags=768
https://nplus1.ru/search?tags=890
https://nplus1.ru/search?tags=871
https://nplus1.ru/search?tags=876
https://nplus1.ru/search?tags=775
https://nplus1.ru/search?tags=767
https://nplus1.ru/search?tags=771
https://nplus1.ru/search?tags=772
https://nplus1.ru/search?tags=778
https://nplus1.ru/search?tags=917
https://nplus1.ru/search?tags=918
https://nplus1.ru/search?tags=824
https://t.me/nplusone
https://vk.com/nplusone
https://ok.ru/nplus1
https://twitter.com/nplusodin
https://nplus1.ru/about
https://nplus1.ru/difficult
https://nplus1.ru/adv
https://nplus1.ru/news/2015/09/21/editor-thy-name
https://nplus1.ru/search?tags=946
https://nplus1.ru/search?tags=8

Ссылок много. Но нам нужны только новости – ссылки, которые начинаются со слова `/news`. Добавим условие: будем выбирать только те ссылки, в которых есть `/news`. Создадим пустой список `urls` и будем добавлять в него только ссылки, которые удовлетворяют этому условию.

In [14]:
urls = []

for link in soup.find_all('a'):
#     print(link)
    if '/news' in link.get('href'):
        if 'https://' in link.get('href'):
            urls.append(link.get('href'))
urls

['https://nplus1.ru/news/2015/09/21/editor-thy-name',
 'https://nplus1.ru/news/2023/12/18/hypochondriasis',
 'https://nplus1.ru/news/2023/12/18/goffins-dunk-food',
 'https://nplus1.ru/news/2023/12/18/daily-toothbrushing-and-pneumonia',
 'https://nplus1.ru/news/2023/12/18/killer-kittens',
 'https://nplus1.ru/news/2023/12/18/metabolic-features-of-depressive-disorder',
 'https://nplus1.ru/news/2023/12/18/hypochondriasis',
 'https://nplus1.ru/news/2023/12/18/parasite-eggs-from-granada',
 'https://nplus1.ru/news/2023/12/18/medieval-deformed-skull',
 'https://nplus1.ru/news/2023/12/18/qafzeh-25',
 'https://nplus1.ru/news/2023/12/16/jwst-bd-light',
 'https://nplus1.ru/news/2023/12/16/hound-record-100m',
 'https://nplus1.ru/news/2023/12/16/retracted-2023',
 'https://nplus1.ru/news/2023/12/16/turn-off',
 'https://nplus1.ru/news/2023/12/16/label-smart',
 'https://nplus1.ru/news/2023/12/16/wisdom-dance',
 'https://nplus1.ru/news/2023/12/15/cr6-forest-fires',
 'https://nplus1.ru/news/2023/12/15/ve

Теперь наша задача сводится к следующему: изучить одну страницу с новостью, научиться из нее вытаскивать текст и всю необходимую информацию, а потом применить весь набор действий к каждой ссылке из `full_urls` в цикле. Посмотрим на новость с индексом 1, у вас может быть другая, новости обновляются.

In [15]:
url0 = urls[1]

page0 = requests.get(url0)
soup0 = BeautifulSoup(page0.text, 'html')
soup0

<!DOCTYPE html>
<html lang="ru" prefix="og: http://ogp.me/ns#">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<title>Ипохондрия повысила риск смерти от всех причин. Особенно от самоубийства</title>
<link as="font" crossorigin="" href="https://staticn1.nplus1.ru/fonts/AeonikPro/AeonikPro-Regular.woff2" rel="preload" type="font/woff2"/>
<link as="font" crossorigin="" href="https://staticn1.nplus1.ru/fonts/Spectral/Spectral-Regular.woff" rel="preload" type="font/woff2"/>
<link href="/front-build/css/main.css?id=9b496fbb252428c03791da99722528ab" rel="stylesheet"/>
<link href="/front-build/css/app.css?id=f9c00b5102d287d01698f43324336ca6" rel="stylesheet"/>
<link href="/apple-touch-icon.png" rel="apple-touch-icon" sizes="180x180"/>
<link href="/favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/>
<link href="/favicon-16x16.png" rel="icon" sizes="16x16" type="image/png"/>
<link href="/site.webmanifest" rel="manifest"/>
<link col

В коде каждой страницы с новостью есть часть с мета-информацией: датой, именем автора и проч. Такая информация окружена тэгом `<meta></meta>`. Посмотрим:

In [16]:
soup0.find_all('meta')

[<meta charset="utf-8"/>,
 <meta content="width=device-width, initial-scale=1" name="viewport"/>,
 <meta content="#f26e40" name="msapplication-TileColor"/>,
 <meta content="#ffffff" name="theme-color"/>,
 <meta content="8c90b02c84ac3b72" name="yandex-verification"/>,
 <meta content="b419949322895fc9106e24ed01be58ac" name="pmail-verification"/>,
 <meta content="N + 1 — главное издание о науке, технике и технологиях" name="description"/>,
 <meta content="N + 1 — главное издание о науке, технике и технологиях" property="og:site_name"/>,
 <meta content="Ипохондрия повысила риск смерти от всех причин" property="og:title"/>,
 <meta content="https://minio.nplus1.ru/app-images/869395/6580200908427_cover_share.jpg" property="og:image"/>,
 <meta content="https://nplus1.ru/news/2023/12/18/hypochondriasis" property="og:url"/>,
 <meta content="N + 1 — главное издание о науке, технике и технологиях" property="og:description"/>,
 <meta content="article" property="og:type"/>,
 <meta content="2023-12-1

Из этого списка нам нужны части с именем автора, датой, заголовком и кратким описанием. Воспользуемся поиском по атрибуту `name`. Передадим функции `find_all()` в качестве аргумента словарь с названием и значением атрибута:

In [19]:
soup0.find_all('meta', {'name' : 'author'}) # например, автор

[<meta content="Слава Гоменюк" name="author"/>]

Теперь выберем единственный элемент полученного списка (с индексом 0):

In [20]:
soup0.find_all('meta', {'name' : 'author'})[0]

<meta content="Слава Гоменюк" name="author"/>

Нам нужно вытащить из этого объекта `content` – имя автора. Посмотрим на атрибуты:

In [24]:
type(soup0.find_all('meta', {'name' : 'author'})[0].attrs)

dict

Как получить отсюда `content`? Очень просто, ведь это словарь! А доставать из словаря значение по ключу мы умеем.

In [23]:
author = soup0.find_all('meta', {'name' : 'author'})[0].get('content')
print(author)

Слава Гоменюк


Аналогичным образом извлечем дату, заголовок и описание.

In [25]:
date = soup0.find_all('meta', {'itemprop' : 'datePublished'})[0].get('content')
title = soup0.find_all('meta', {'property' : 'og:title'})[0].get('content')

In [27]:
date

'2023-12-18'

In [44]:
from datetime import datetime

In [45]:
from datetime import time

In [50]:
date_format = '%Y-%m-%d'

In [51]:
news_date = datetime.strptime(date, date_format)

In [52]:
news_date

datetime.datetime(2023, 12, 18, 0, 0)

In [56]:
delta = datetime.now() - news_date

In [ ]:
dir(delta)

In [58]:
delta.days

0

In [63]:
date_format1 = '%d.%m.%Y-%H:%M:%S'

In [64]:
datetime.now().strftime(date_format1)

'18.12.2023-18:30:15'

In [26]:
title

'Ипохондрия повысила риск смерти от\xa0всех причин'

Теперь осталось совсем чуть-чуть. Написать готовую функцию для всех проделанных нами действий и применить ее в цикле для всех ссылок в списке `full_urls`. Напишем! Аргументом функции будет ссылка на новость, а возвращать она будет текст новости и всю необходимую информацию (дата, автор, сложность и проч.). Скопируем все строки кода выше.

In [30]:
def GetNews(url0):
    """
    Возвращает кортеж с url0, date, author, title.
    Параметры:

    url0 - ссылка на новость (строка).
    """
    page0 = requests.get(url0)
    soup0 = BeautifulSoup(page0.text, 'html')

    try:
        author = soup0.find_all('meta', {'name' : 'author'})[0].get('content')
    except IndexError:
        author = None
    date = soup0.find_all('meta', {'itemprop' : 'datePublished'})[0].get('content')
    title = soup0.find_all('meta', {'property' : 'og:title'})[0].get('content')


    return url0, date, author, title

In [31]:
help(GetNews)

Help on function GetNews in module __main__:

GetNews(url0)
    Возвращает кортеж с url0, date, author, title.
    Параметры:
    
    url0 - ссылка на новость (строка).



Уфф. Осталось применить ее в цикле. Но давайте не будем спешить: импортируем функцию `sleep` для задержки, чтобы на каждой итерации цикла, прежде чем перейти к следующей новости, Python ждал несколько секунд. Во-первых, это нужно, чтобы сайт «не понял», чтобы мы его грабим, да еще автоматически. Во-вторых, с небольшой задержкой всегда есть гарантия, что страница прогрузится (сейчас это пока не очень важно, но особенно актуально будет, когда будем обсуждать встраивание в браузер с Selenium). Приступим.

In [32]:
from time import sleep

In [34]:
len(urls)

52

In [35]:
news = [] # это будет список из кортежей, в которых будут храниться данные по каждой новости

for link in urls[20:22]:
    print(link)
    res = GetNews(link)
    news.append(res)

    sleep(3) # задержка в 3 секунды

https://nplus1.ru/news/2023/12/15/cobalt-catalyzed-borylation
https://nplus1.ru/news/2023/12/15/domestic-yak


In [36]:
news[1]

('https://nplus1.ru/news/2023/12/15/domestic-yak',
 '2023-12-15',
 'Михаил Подрезов',
 'Жители Тибета разводили домашних яков не\xa0меньше 2500 лет назад')

Так теперь выглядит первый элемент списка:

Импортируем `pandas` и создадим датафрейм из списка кортежей:

In [37]:
import pandas as pd

In [38]:
df = pd.DataFrame(news)

In [41]:
df.head(11)

,link,date,author,title
0,https://nplus1.ru/news/2023/12/15/cobalt-catal...,2023-12-15,Михаил Бойм,Кобальтовый катализатор помог провести реакцию...
1,https://nplus1.ru/news/2023/12/15/domestic-yak,2023-12-15,Михаил Подрезов,Жители Тибета разводили домашних яков не меньш...


Переименуем столбцы в базе.

In [40]:
df.columns = ['link', 'date', 'author', 'title']

In [42]:
df.head(2)

,link,date,author,title
0,https://nplus1.ru/news/2023/12/15/cobalt-catal...,2023-12-15,Михаил Бойм,Кобальтовый катализатор помог провести реакцию...
1,https://nplus1.ru/news/2023/12/15/domestic-yak,2023-12-15,Михаил Подрезов,Жители Тибета разводили домашних яков не меньш...


Всё! Сохраняем датафрейм в файл. Для разнообразия сохраним в Excel:

In [43]:
df.to_excel('nplus-news.xlsx', index=False)